# Dataprep

### Objective

Crawls through raw_data directory and converts diffusion and T2EG into a data array 

### Prerequisites

All diffusion and T2EG should be registrated and put in a NIFTI file format.

### Data organisation

- All b0 diffusion should be named "patientid_sX_DWIb0.nii.gz" where "sX" corresponds to time delay and can be "s0" or "s1" (to stratify on stroke/hemorrhage)
- All b1000 diffusion should be named "patientid_sX_DWIb1000.nii.gz" where "sX" corresponds to time delay and can be "s0" or "s1" (to stratify on stroke/hemorrhage)
- All corresponding T2EG sequences should be named: "patientid_sX_qX_t2eg.nii.gz" where "qX" corresponds to quality and can be "q0" to "q3" (to stratify on quality)
- A mask "patientid_hX_MASK.nii.gz" with 3 binary dimensions : 0 (brain mask) and 1 (stroke region) and 2 (hemorrhage region)

## Load modules

In [ ]:
import os, glob, h5py
import numpy as np
from skimage.morphology import dilation, opening
from modules.niftitools import twoniftis2array, t2egnifti2array, masknifti2array

## Crawl through files

In [ ]:
dwifiles_precheck = glob.glob(os.path.join("raw_data", "*_DWIb0.nii.gz"))
patnames, strokehemorrs, qualities, b0files, b1000files, t2egfiles, maskfiles = [], [], [], [], [], [], []
num_patients = 0
total_slices = 0
for dwifile in dwifiles_precheck:
    name, strokehemorr, _ = os.path.basename(dwifile).split("_")
    strokehemorr = int(timepoint.replace("s",""))
    matchesb1000 = glob.glob(os.path.join("raw_data", name+"_h"+str(timepoint)+"_DWIb1000.nii.gz"))
    matchest2eg = glob.glob(os.path.join("raw_data", name+"_h"+str(timepoint)+"_q*_t2eg.nii.gz"))
    if len(matchest2eg) and len(matchesb1000):
        _, _, quality, _ = os.path.basename(matchest2eg[0]).split("_")
        patnames.append(name)
        strokehemorrs.append(timepoint)
        qualities.append(int(quality.replace("q","")))
        b0files.append(dwifile)
        b1000files.append(matchesb1000[0])
        t2egfiles.append(matchest2eg[0])
        matchesMask = glob.glob(os.path.join("raw_data", name+"_h"+str(timepoint)+"_MASK.nii.gz"))
        if len(matchesMask):
            maskfiles.append(matchesMask[0])
        else:
            maskfiles.append(None)
        num_patients += 1
        total_slices += np.squeeze(nib.load(dwifile).get_fdata()).shape[2]

## Create data arrays

In [ ]:

outputdir = "data"

In [ ]:
with h5py.File(os.path.join(outputdir,"metadata.hdf5"), "w") as metadata:
    metadata.create_dataset("patientnames", data=np.array(patnames, dtype="S"))
    metadata.create_dataset("shape_x", data=(total_slices,256,256,3))
    metadata.create_dataset("shape_y", data=(total_slices,256,256,1))
    metadata.create_dataset("shape_mask", data=(total_slices,256,256,3))
    metadata.create_dataset("shape_meta", data=(total_slices,4))

In [ ]:
fx = np.memmap(os.path.join(outputdir,"data_x.dat"), dtype="float32", mode="w+",
               shape=(z_slices,256,256,3))
fy = np.memmap(os.path.join(outputdir,"data_y.dat"), dtype="float32", mode="w+",
               shape=(z_slices,256,256,1))
fmask = np.memmap(os.path.join(outputdir,"data_mask.dat"), dtype="int8", mode="w+",
               shape=(z_slices,256,256,1))
fmeta = np.memmap(os.path.join(outputdir,"data_meta.dat"), dtype="float32", mode="w+",
               shape=(z_slices,4))

if num_patients > 0:
    print("Imported following patients:", end=" ")

current_begin_slice = 0
for i in range(num_patients):
    if i>0:
        print(", ",end="")
    Xdata, mask, _ = twoniftis2array(b0files[i], b1000files[i],z_slices)
    curlen = Xdata.shape[2]
    Xdata = Xdata.transpose(1,2,3,0)
    fx[current_begin_slice+curlen] = Xdata
    if maskfiles[i] is not None:
        fmask[current_begin_slice+curlen] = masknifti2array(maskfiles[i])[...,np.newaxis]
    else:
        crudemask = dilation(dilation(dilation(opening(np.logical_and(mask, Xdata[...,2]<600)))))
        crudemask = crudemask.astype("int8") + mask.astype("int8")
        fmask[current_begin_slice+curlen,0] = mask[...,np.newaxis]
        fmask[current_begin_slice+curlen,1] = crudemask[...,np.newaxis]
    fy[current_begin_slice+curlen] = t2egnifti2array(t2egfiles[i],mask,z_slices)[...,np.newaxis]
    fmeta[current_begin_slice+curlen,0] = qualities[i]
    fmeta[current_begin_slice+curlen,1] = np.arange(curlen)
    fmeta[current_begin_slice+curlen,2] = qualities[i]
    fmeta[current_begin_slice+curlen,3] = timepoints[i]
    print(name, end="")
    
del fx, fy, fmask, fmeta